In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from keplergl import KeplerGl
from pyproj import CRS

In [2]:
fp = r'data/bike-stations.shp'
geodata = gpd.read_file(fp)

geodata.head()

,FID,ID,Nimi,Namn,Name,Osoite,Adress,Kaupunki,Stad,Operaattor,Kapasiteet,x,y,geometry
0,1,501,Hanasaari,Hanaholmen,Hanasaari,Hanasaarenranta 1,Hanaholmsstranden 1,Espoo,Esbo,CityBike Finland,10,24.840319,60.165820,POINT (24.84032 60.16582)
1,2,503,Keilalahti,Kägelviken,Keilalahti,Keilalahdentie 2,Kägelviksvägen 2,Espoo,Esbo,CityBike Finland,28,24.827467,60.171524,POINT (24.82747 60.17152)
2,3,505,Westendinasema,Westendstationen,Westendinasema,Westendintie 1,Westendvägen 1,Espoo,Esbo,CityBike Finland,16,24.805758,60.168266,POINT (24.80576 60.16827)
3,4,507,Golfpolku,Golfstigen,Golfpolku,Golfpolku 3,Golfstigen 3,Espoo,Esbo,CityBike Finland,16,24.796136,60.168143,POINT (24.79614 60.16814)
4,5,509,Revontulentie,Norrskensvägen,Revontulentie,Revontulentie 10,Norrskensvägen 10,Espoo,Esbo,CityBike Finland,30,24.802938,60.171551,POINT (24.80294 60.17155)


In [3]:
print(geodata.crs.name)
print(len(geodata))
geodata.plot(markersize=1);

WGS 84
457


In [4]:
bikefp = r'data/2019-07.csv'
data = pd.read_csv(bikefp)

data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32,Eläinmuseo,211.0,Haagan tori,5211.0,37893
1,2019-07-31T23:59:55,2019-08-01T00:05:15,32,Eläinmuseo,36.0,Apollonkatu,939.0,315
2,2019-07-31T23:59:48,2019-08-01T00:04:53,101,Munkkivuoren ostoskeskus,100.0,Teljäntie,994.0,301
3,2019-07-31T23:59:42,2019-08-01T00:06:55,109,Hertanmäenkatu,142.0,Koskelantie,1827.0,428
4,2019-07-31T23:59:40,2019-08-01T00:07:26,118,Fleminginkatu,111.0,Esterinportti,1833.0,461


In [5]:
print(len(data))

666362


In [6]:
geodata['ID'] = geodata['ID'].astype(int)
geodata_origin = geodata.add_suffix('_origin')

In [7]:
# creating a data with no aggregation
coor_data = data.merge(geodata_origin, left_on='Departure station id', right_on='ID_origin', how='outer')

coor_data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),FID_origin,ID_origin,...,Name_origin,Osoite_origin,Adress_origin,Kaupunki_origin,Stad_origin,Operaattor_origin,Kapasiteet_origin,x_origin,y_origin,geometry_origin
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32.0,Eläinmuseo,211.0,Haagan tori,5211.0,37893.0,142.0,32.0,...,Finnish Museum of Natural History,Pohjoinen Rautatiekatu 13,Norra Järnvägsgatan 13,None,None,None,20.0,24.931898,60.171359,POINT (24.93190 60.17136)
1,2019-07-31T23:59:55,2019-08-01T00:05:15,32.0,Eläinmuseo,36.0,Apollonkatu,939.0,315.0,142.0,32.0,...,Finnish Museum of Natural History,Pohjoinen Rautatiekatu 13,Norra Järnvägsgatan 13,None,None,None,20.0,24.931898,60.171359,POINT (24.93190 60.17136)
2,2019-07-31T23:59:36,2019-08-01T00:31:30,32.0,Eläinmuseo,211.0,Haagan tori,5452.0,1908.0,142.0,32.0,...,Finnish Museum of Natural History,Pohjoinen Rautatiekatu 13,Norra Järnvägsgatan 13,None,None,None,20.0,24.931898,60.171359,POINT (24.93190 60.17136)
3,2019-07-31T20:57:35,2019-07-31T21:13:11,32.0,Eläinmuseo,36.0,Apollonkatu,1208.0,931.0,142.0,32.0,...,Finnish Museum of Natural History,Pohjoinen Rautatiekatu 13,Norra Järnvägsgatan 13,None,None,None,20.0,24.931898,60.171359,POINT (24.93190 60.17136)
4,2019-07-31T20:56:17,2019-07-31T21:07:30,32.0,Eläinmuseo,40.0,Hakaniemi (M),2060.0,672.0,142.0,32.0,...,Finnish Museum of Natural History,Pohjoinen Rautatiekatu 13,Norra Järnvägsgatan 13,None,None,None,20.0,24.931898,60.171359,POINT (24.93190 60.17136)


In [8]:
geodata_destin = geodata.add_suffix('_destin')

In [9]:
# creating a data with no aggregation
coor_data = coor_data.merge(geodata_destin, left_on='Return station id', right_on='ID_destin', how='outer')

coor_data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),FID_origin,ID_origin,...,Name_destin,Osoite_destin,Adress_destin,Kaupunki_destin,Stad_destin,Operaattor_destin,Kapasiteet_destin,x_destin,y_destin,geometry_destin
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32.0,Eläinmuseo,211.0,Haagan tori,5211.0,37893.0,142.0,32.0,...,Haagan tori,Haagan tori 1,Haga torg 1,None,None,None,16.0,24.89893,60.212112,POINT (24.89893 60.21211)
1,2019-07-31T23:59:36,2019-08-01T00:31:30,32.0,Eläinmuseo,211.0,Haagan tori,5452.0,1908.0,142.0,32.0,...,Haagan tori,Haagan tori 1,Haga torg 1,None,None,None,16.0,24.89893,60.212112,POINT (24.89893 60.21211)
2,2019-07-31T17:20:33,2019-07-31T17:31:12,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,1641.0,628.0,210.0,101.0,...,Haagan tori,Haagan tori 1,Haga torg 1,None,None,None,16.0,24.89893,60.212112,POINT (24.89893 60.21211)
3,2019-07-28T13:20:37,2019-07-28T13:28:30,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,1642.0,471.0,210.0,101.0,...,Haagan tori,Haagan tori 1,Haga torg 1,None,None,None,16.0,24.89893,60.212112,POINT (24.89893 60.21211)
4,2019-07-27T12:03:49,2019-07-27T12:29:01,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,1858.0,1507.0,210.0,101.0,...,Haagan tori,Haagan tori 1,Haga torg 1,None,None,None,16.0,24.89893,60.212112,POINT (24.89893 60.21211)


In [10]:
coor_data.dtypes

Departure                   object
Return                      object
Departure station id       float64
Departure station name      object
Return station id          float64
Return station name         object
Covered distance (m)       float64
Duration (sec.)            float64
FID_origin                 float64
ID_origin                  float64
Nimi_origin                 object
Namn_origin                 object
Name_origin                 object
Osoite_origin               object
Adress_origin               object
Kaupunki_origin             object
Stad_origin                 object
Operaattor_origin           object
Kapasiteet_origin          float64
x_origin                   float64
y_origin                   float64
geometry_origin           geometry
FID_destin                 float64
ID_destin                  float64
Nimi_destin                 object
Namn_destin                 object
Name_destin                 object
Osoite_destin               object
Adress_destin       

In [11]:
coor_data = coor_data[['Departure','Return', 'Departure station id', 'Departure station name' , 'Return station id',
                      'Return station name', 'x_origin', 'y_origin', 'x_destin',  'y_destin', 'geometry_origin']]


In [12]:
coor_data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,x_origin,y_origin,x_destin,y_destin,geometry_origin
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136)
1,2019-07-31T23:59:36,2019-08-01T00:31:30,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136)
2,2019-07-31T17:20:33,2019-07-31T17:31:12,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)
3,2019-07-28T13:20:37,2019-07-28T13:28:30,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)
4,2019-07-27T12:03:49,2019-07-27T12:29:01,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)


In [13]:
agg_data = coor_data.copy()

In [14]:
# aggregating data
# creating pairs
# if 457 station, then we make a lot of pairs.
origins = geodata['ID'].to_list()
destins = geodata['ID'].to_list()

# pairs = pd.DataFrame()

pairs_list=[]

for origin in origins:
    for destin in destins:
        stat_pair = (int(origin), int(destin))
        
        pairs_list.append(stat_pair)
        

routes_list = list(range(len(pairs_list)))

# pairs_list

In [15]:
routes_dict = dict(zip(pairs_list, routes_list))


In [16]:
for i in pairs_list:
    print(i, routes_dict[i])
    
    break

(501, 501) 0


In [17]:
agg_data = agg_data.dropna()

agg_data['pairs'] = [(int(orID), int(deID)) for orID, deID in zip(agg_data['Departure station id'].to_list(),
                                                                  agg_data['Return station id'].to_list())]
agg_data.head()                                                                 

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,x_origin,y_origin,x_destin,y_destin,geometry_origin,pairs
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136),"(32, 211)"
1,2019-07-31T23:59:36,2019-08-01T00:31:30,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136),"(32, 211)"
2,2019-07-31T17:20:33,2019-07-31T17:31:12,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598),"(101, 211)"
3,2019-07-28T13:20:37,2019-07-28T13:28:30,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598),"(101, 211)"
4,2019-07-27T12:03:49,2019-07-27T12:29:01,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598),"(101, 211)"


In [18]:
agg_data = agg_data.reset_index(drop=True)

# add route id
agg_data['route_id'] = None

for i in list(range(len(agg_data))):
    pair_route = agg_data.at[i, 'pairs']
    
    route_id =  routes_dict[pair_route]
    
    agg_data.at[i, 'route_id'] = route_id
    
    
    
agg_data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,x_origin,y_origin,x_destin,y_destin,geometry_origin,pairs,route_id
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136),"(32, 211)",64709
1,2019-07-31T23:59:36,2019-08-01T00:31:30,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136),"(32, 211)",64709
2,2019-07-31T17:20:33,2019-07-31T17:31:12,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598),"(101, 211)",95785
3,2019-07-28T13:20:37,2019-07-28T13:28:30,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598),"(101, 211)",95785
4,2019-07-27T12:03:49,2019-07-27T12:29:01,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598),"(101, 211)",95785


In [19]:
# aggregating movers
movers = gpd.GeoDataFrame(index = agg_data['route_id'].unique())

for key, group in agg_data.groupby('route_id'):
    
    movers.at[key, 'route_id'] = key
    
    movers.at[key, 'movers'] = int(len(group))
    
    movers.at[key, 'origin_id'] = group['Departure station id'].unique()
    movers.at[key, 'origin_name'] = group['Departure station name'].unique()[0]
    movers.at[key, 'destin_id'] = group['Return station id'].unique()
    movers.at[key, 'destin_name'] = group['Return station name'].unique()[0]
    
    movers.at[key, 'x_origin'] = group['x_origin'].unique()[0]
    movers.at[key, 'y_origin'] = group['y_origin'].unique()[0]
    movers.at[key, 'x_destin'] = group['x_destin'].unique()[0]
    movers.at[key, 'y_destin'] = group['y_destin'].unique()[0]
    
#     movers.at[key, 'geometry'] = group['geometry_origin'].unique()[0]
           
    
movers.head()

,route_id,movers,origin_id,origin_name,destin_id,destin_name,x_origin,y_origin,x_destin,y_destin
64709,64709.0,2.0,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112
95785,95785.0,21.0,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112
98984,98984.0,9.0,109.0,Hertanmäenkatu,211.0,Haagan tori,24.922331,60.202694,24.89893,60.212112
103097,103097.0,16.0,118.0,Fleminginkatu,211.0,Haagan tori,24.952207,60.189525,24.89893,60.212112
71107,71107.0,3.0,46.0,Diakoniapuisto,211.0,Haagan tori,24.943541,60.186904,24.89893,60.212112


In [20]:
movers['geometry'] = [Point(originx, originy) for originx, originy in zip(movers['x_origin'].to_list(),
                                                                          movers['y_origin'].to_list())]
movers.head()

,route_id,movers,origin_id,origin_name,destin_id,destin_name,x_origin,y_origin,x_destin,y_destin,geometry
64709,64709.0,2.0,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136)
95785,95785.0,21.0,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)
98984,98984.0,9.0,109.0,Hertanmäenkatu,211.0,Haagan tori,24.922331,60.202694,24.89893,60.212112,POINT (24.92233 60.20269)
103097,103097.0,16.0,118.0,Fleminginkatu,211.0,Haagan tori,24.952207,60.189525,24.89893,60.212112,POINT (24.95221 60.18953)
71107,71107.0,3.0,46.0,Diakoniapuisto,211.0,Haagan tori,24.943541,60.186904,24.89893,60.212112,POINT (24.94354 60.18690)


_________________________________________________

In [21]:
# coor_data for visualization
# movers for aggregated visualization

In [22]:
movers.crs = CRS.from_epsg(4326)

In [23]:
movers.crs.name

'WGS 84'

In [24]:
coor_data = gpd.GeoDataFrame(coor_data, geometry='geometry_origin')

coor_data = coor_data.rename(columns={'geometry_origin': 'geometry'})

coor_data = coor_data.set_geometry('geometry')

In [25]:
coor_data['geometry'] = [Point(originx, originy) for originx, originy in zip(coor_data['x_origin'].to_list(),
                                                                          coor_data['y_origin'].to_list())]

In [26]:
coor_data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,x_origin,y_origin,x_destin,y_destin,geometry
0,2019-07-31T23:59:56,2019-08-01T10:31:34,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136)
1,2019-07-31T23:59:36,2019-08-01T00:31:30,32.0,Eläinmuseo,211.0,Haagan tori,24.931898,60.171359,24.89893,60.212112,POINT (24.93190 60.17136)
2,2019-07-31T17:20:33,2019-07-31T17:31:12,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)
3,2019-07-28T13:20:37,2019-07-28T13:28:30,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)
4,2019-07-27T12:03:49,2019-07-27T12:29:01,101.0,Munkkivuoren ostoskeskus,211.0,Haagan tori,24.879807,60.205983,24.89893,60.212112,POINT (24.87981 60.20598)


In [27]:
coor_data.crs = CRS.from_epsg(4326)

In [28]:
movers.crs.name

'WGS 84'

In [29]:
coor_data.dtypes
coor_data['Departure'] = coor_data['Departure'].astype(str)

In [30]:
coor_data = coor_data[['Departure','Departure station name', 'Return station name', 'x_origin', 'y_origin', 'x_destin', 'y_destin', 'geometry']]
type(coor_data)

geopandas.geodataframe.GeoDataFrame

In [31]:
movers.dtypes

route_id        float64
movers          float64
origin_id       float64
origin_name      object
destin_id       float64
destin_name      object
x_origin        float64
y_origin        float64
x_destin        float64
y_destin        float64
geometry       geometry
dtype: object

In [32]:
movers = movers[['route_id', 'movers', 'x_origin', 'y_origin', 'x_destin', 'y_destin', 'geometry' ]]
type(movers)


geopandas.geodataframe.GeoDataFrame

__________________________________________

In [33]:
# Create KeplerGl instance
m = KeplerGl(height=600)
moves = KeplerGl(height=600)

# Add stop duration
m.add_data(coor_data, 'individual movements')
moves.add_data(movers, 'agg movements')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [34]:
# m

In [36]:
# Save map as html
m.save_to_html(file_name='root/individual-movements.html', config=config, read_only=True)

Map saved to root/individual-movements.html!


In [38]:
# Save map as html
# moves.save_to_html(file_name='index.html', config=config, read_only=True)
moves.save_to_html(file_name='index.html', config=config, read_only=False)

Map saved to index.html!


In [43]:
moves.save_to_html(file_name='root/3d-map.html', config=config3d, read_only=True)

Map saved to root/3d-map.html!
